<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Sklearn" data-toc-modified-id="Sklearn-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Sklearn</a></span><ul class="toc-item"><li><span><a href="#sklearn.grid_search" data-toc-modified-id="sklearn.grid_search-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>sklearn.grid_search</a></span><ul class="toc-item"><li><span><a href="#Генерация-датасета" data-toc-modified-id="Генерация-датасета-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Генерация датасета</a></span></li><li><span><a href="#Задание-модели" data-toc-modified-id="Задание-модели-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Задание модели</a></span></li><li><span><a href="#Генерация-сетки" data-toc-modified-id="Генерация-сетки-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>Генерация сетки</a></span></li><li><span><a href="#Подбор-параметров-и-оценка-качества" data-toc-modified-id="Подбор-параметров-и-оценка-качества-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>Подбор параметров и оценка качества</a></span><ul class="toc-item"><li><span><a href="#Grid-search" data-toc-modified-id="Grid-search-1.1.4.1"><span class="toc-item-num">1.1.4.1&nbsp;&nbsp;</span>Grid search</a></span></li><li><span><a href="#Randomized-grid-search" data-toc-modified-id="Randomized-grid-search-1.1.4.2"><span class="toc-item-num">1.1.4.2&nbsp;&nbsp;</span>Randomized grid search</a></span></li></ul></li></ul></li></ul></li></ul></div>

**Корректность проверена на Python 3.6:**
+ pandas 0.23.4
+ numpy 1.15.4
+ sklearn 0.20.2

каким образом можно подбирать параметры, оптимальные для решаемой задачи? В этом видео мы будем рассматривать модуль grid_search в библиотеке sklearn и научимся подбирать параметры модели по сетке. 

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Sklearn

## sklearn.grid_search

документация: http://scikit-learn.org/stable/modules/grid_search.html

In [2]:
from sklearn import model_selection, datasets, linear_model, metrics

import numpy as np
import pandas as pd

### Генерация датасета

In [3]:
iris = datasets.load_iris()

In [12]:
model_selection.train_test_split?

In [4]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(iris.data, iris.target, 
                                                                                     test_size = 0.3,random_state = 0)

### Задание модели

давайте выберем модель — пусть это будет SGD-классификатор — и создадим объект с параметрами по умолчанию. Теперь можно подбирать параметры. 

In [5]:
linear_model.SGDClassifier?

In [6]:
classifier = linear_model.SGDClassifier(random_state = 0, tol=1e-3)

### Генерация сетки

In [7]:
classifier.get_params().keys()

dict_keys(['alpha', 'average', 'class_weight', 'early_stopping', 'epsilon', 'eta0', 'fit_intercept', 'l1_ratio', 'learning_rate', 'loss', 'max_iter', 'n_iter_no_change', 'n_jobs', 'penalty', 'power_t', 'random_state', 'shuffle', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

1. Можно выбрать вид функции потерь, будем рассматривать hinge, log, squared_hinge и squared_loss. 
2. Также давайте выберем вид регуляризации — выберем между l1 и l2.
3. Также можем подобрать количество итераций — давайте подбирать от 5 до 10, по умолчанию у нас 5 итераций. 
5. И выберем коэффициент alpha — это множитель перед регуляризацией. По умолчанию у нас доступно значение 0,0001. Ну, вот давайте создадим отрезок от 0,0001 до 0,001, бросим на него равномерно пять точек и будем использовать их в качестве весов.


In [13]:
parameters_grid = {
    'loss' : ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty' : ['l1', 'l2'],
    'max_iter' : np.arange(5,10),
    'alpha' : np.linspace(0.0001, 0.001, num = 5),
}

Итак, давайте создадим стратегию кросс-валидации, с помощью которой мы будем оценивать качество. В данном случае я использую `StratifiedShuffleSplit`, будем делать 10 разбиений, и в тестовую выборку будет идти 20 % данных. И теперь можно перейти непосредственно к подбору параметров. 

In [14]:
model_selection.StratifiedShuffleSplit?

In [15]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 0)

### Подбор параметров и оценка качества

#### Grid search

 В данном случае в **качестве сетки мы будем использовать словарь `parameters_grid`, у которого ключ — это название параметра, который мы подбираем, а в качестве значения идет набор значений, которые мы хотим проверить**. Таким образом, если мы построим декартово произведение на этих параметрах, то мы получим точки со всеми возможными наборами параметров. Собственно, это нам и хочется получить. **Мы хотим в каждой из этих точек измерить качество классификации, далее сравнить, посмотреть, где качество максимальное, и сказать, что вот это есть оптимальные параметры**.
 
 Для начала давайте **создадим объект grid_search, с помощью которого мы будем это делать**. Данному **объекту нужно обязательно передать модель, которую мы хотим оптимизировать** — в данном случае наш SGD-классификатор. Также нужно **передать сетку с параметрами**, по которой мы будем бегать. Нужно **указать метрику, которую мы будем проверять**, и **стратегию кросс-валидации**.

In [ ]:
model_selection.GridSearchCV?

In [16]:
grid_cv = model_selection.GridSearchCV(estimator=classifier,
                                       param_grid=parameters_grid,
                                       scoring = 'accuracy',
                                       cv = cv)

In [17]:
%%time
grid_cv.fit(train_data, train_labels)

Wall time: 4.84 s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             estimator=SGDClassifier(random_state=0),
             param_grid={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                         'loss': ['hinge', 'log', 'squared_hinge',
                                  'squared_loss'],
                         'max_iter': array([5, 6, 7, 8, 9]),
                         'penalty': ['l1', 'l2']},
             scoring='accuracy')

**нас интересует самый лучший классификатор — его можно найти с помощью команды `best_estimator`**. Видим, что здесь нам возвращается модель с лучшими параметрами. Отдельно можем попросить best_score, или оценку на лучшем наборе параметров, и, собственно, вывести лучшие наборы параметров в виде вот просто словаря параметров.

In [18]:
grid_cv.best_estimator_

SGDClassifier(alpha=0.0007750000000000001, max_iter=9, penalty='l1',
              random_state=0)

In [19]:
print(grid_cv.best_score_)
print(grid_cv.best_params_)

0.9047619047619048
{'alpha': 0.0007750000000000001, 'loss': 'hinge', 'max_iter': 9, 'penalty': 'l1'}


In [20]:
grid_cv.cv_results_

{'mean_fit_time': array([0.0021975 , 0.00159557, 0.00159581, 0.00159497, 0.00159562,
        0.00159585, 0.00159566, 0.00169556, 0.00159569, 0.00159574,
        0.00149601, 0.00169544, 0.00159574, 0.00179517, 0.00170028,
        0.00159569, 0.00179515, 0.00159571, 0.0017952 , 0.00159643,
        0.00169663, 0.00149598, 0.00159574, 0.00149605, 0.00169549,
        0.00169563, 0.00179527, 0.00159569, 0.00169547, 0.00179493,
        0.00179517, 0.00159569, 0.00149593, 0.00149603, 0.00169532,
        0.00159578, 0.00149591, 0.00159583, 0.00159566, 0.00149593,
        0.00159564, 0.00159559, 0.00169537, 0.00159578, 0.00179491,
        0.00159557, 0.00169539, 0.00149369, 0.00159497, 0.00149634,
        0.00169351, 0.00159478, 0.00210075, 0.00199702, 0.0018873 ,
        0.00159569, 0.00169556, 0.00159581, 0.00169959, 0.00179608,
        0.0015964 , 0.00169981, 0.00169859, 0.00168855, 0.0015959 ,
        0.00169561, 0.00169148, 0.00208964, 0.00170665, 0.00159707,
        0.00169663, 0.00170348,

#### Randomized grid search

**Если бы мы работали с большим набором данных, где было бы много признаков и много объектов, каждая модель обучалась бы значительное количество времени и мы бы хотели действительно подобрать много параметров**. Тогда этот процесс занял бы существенное время, и с точки зрения вычислительной эффективности, возможно, нам было бы не так выгодно использовать полный перебор по сетке. Какие есть альтернативы? **Можно использовать случайный поиск по сетке**. 

In [24]:
model_selection.RandomizedSearchCV?

In [25]:
randomized_grid_cv = model_selection.RandomizedSearchCV(estimator=classifier,
                                                        param_distributions=parameters_grid,
                                                        scoring = 'accuracy', cv = cv, n_iter = 20, 
                                                        random_state = 0)

In [26]:
%%time
randomized_grid_cv.fit(train_data, train_labels)

Wall time: 470 ms


RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
                   estimator=SGDClassifier(random_state=0), n_iter=20,
                   param_distributions={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                                        'loss': ['hinge', 'log',
                                                 'squared_hinge',
                                                 'squared_loss'],
                                        'max_iter': array([5, 6, 7, 8, 9]),
                                        'penalty': ['l1', 'l2']},
                   random_state=0, scoring='accuracy')

In [27]:
print(randomized_grid_cv.best_score_)
print(randomized_grid_cv.best_params_)

0.8666666666666666
{'penalty': 'l1', 'max_iter': 8, 'loss': 'squared_hinge', 'alpha': 0.0007750000000000001}


In [30]:
#!/usr/bin/python

from datetime import datetime


def control_sum(str):
    return sum(map(ord, list(str)))


def check(reply):
    def _is_number(str):
        try:
            int(str)
            return True
        except ValueError:
            return False

    if "Control sum:" not in reply:
        return False
    parts = reply.split("Control sum:")
    received_current_time = parts[0].strip()
    received_control_sum = parts[1].strip()
    if not _is_number(received_control_sum):
        return False
    else:
        received_control_sum = int(received_control_sum)
    expected_control_sum = control_sum(received_current_time)
    return expected_control_sum == received_control_sum


current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(current_time)
print("Control sum: " + str(control_sum(current_time)))

#print check(current_time + '\n' + "Control sum: " + str(control_sum(current_time)))

2021-07-30 21:10:58
Control sum: 942
